In [1]:
import sys
sys.path.append('..')

In [2]:
import time
from gerumo.data.dataset import describe_dataset
from gerumo.data.generators import build_generator
from gerumo.utils.engine import (
    setup_cfg, setup_environment, setup_experiment, setup_model,
    build_dataset, build_callbacks, build_metrics, build_optimizer, build_loss
)
from gerumo.models.base import build_model
from gerumo.visualization.metrics import training_history


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

## Select configuration

In [3]:
args['config_file'] = '/home/ir-riqu1/gerumo2/config/cnn_classification.yml'
args['opts'] = []

## Setup

In [4]:
cfg = setup_cfg(args)
output_dir = setup_experiment(cfg)
logger = setup_environment(cfg)

## Load Datasets

In [5]:
train_dataset = build_dataset(cfg, 'train')
describe_dataset(train_dataset, logger,
                save_to=output_dir / "train_description.txt")
validation_dataset = build_dataset(cfg, 'validation')
describe_dataset(validation_dataset, logger,
                save_to=output_dir / "validation_description.txt")

[GERUMO]: INFO     files: 20
[GERUMO]: INFO     events: 199359
[GERUMO]: INFO     observations: 1244338
[GERUMO]: INFO     obsevation by telescopes
[GERUMO]: INFO     NectarCam    427298
FlashCam     387214
CHEC         373163
LSTCam        56663
Name: camera_type, dtype: int64
[GERUMO]: INFO     files: 20
[GERUMO]: INFO     events: 199359
[GERUMO]: INFO     observations: 1244338
[GERUMO]: INFO     obsevation by telescopes
[GERUMO]: INFO     NectarCam    427298
FlashCam     387214
CHEC         373163
LSTCam        56663
Name: camera_type, dtype: int64


## Build generators

In [6]:
train_generator = build_generator(cfg, train_dataset)
validation_generator = build_generator(cfg, validation_dataset)

## Build model

In [9]:
input_shape = train_generator.get_input_shape()
model = build_model(cfg, input_shape)

## Build training tools

In [11]:
callbacks = build_callbacks(cfg)
metrics = build_metrics(cfg)
optimizer = build_optimizer(cfg)
loss = build_loss(cfg)

2021-12-31 20:57:31.549791: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-31 20:57:31.549811: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-31 20:57:31.899434: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


## Compile model

In [12]:
model = setup_model(
    model, train_generator, optimizer, loss, metrics
)

Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          [(64, 55, 47, 3)]         0         
_________________________________________________________________
conv2d_10 (Conv2D)           multiple                  1792      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
conv_block_4 (ConvBlock)     multiple                  615680    
_________________________________________________________________
conv_block_5 (ConvBlock)     multiple                  887296    
_________________________________________________________________
conv_block_6 (ConvBlock)     multiple                  3544064   
_________________________________________________________________
conv_block_7 (ConvBlock)     multiple                  141660

## Start training

In [ ]:
train_generator.fit_mode()
validation_generator.fit_mode()
model.fit_mode()

In [ ]:
start_time = time.time()
history = model.fit(
    train_generator,
    epochs= cfg.SOLVER.EPOCHS,
    verbose=2,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks,
    use_multiprocessing=False,
    workers=1,
    max_queue_size=20,
)
training_time = (time.time() - start_time)/60.0

In [ ]:
logger.info(f"Training time: {training_time:.3f} [min]")

In [ ]:
training_history(history, training_time, cfg.EXPERIMENT_NAME)

In [ ]:
training_history(history, training_time, cfg.EXPERIMENT_NAME, ylog=True)